<a href="https://colab.research.google.com/github/soichi-fujiwara/jupyter-notebook/blob/master/%E9%9F%B3%E6%A5%BD%E3%82%B0%E3%83%AB%E3%83%BC%E3%83%97%E5%8F%96%E5%BE%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install beautifulsoup4
!pip install mecab-python3

     |████████████████████████████████| 15.9MB 39.8MB/s 


In [0]:
import MeCab
import re
import requests
from bs4 import BeautifulSoup

In [28]:
### データの用意 ###
# Wikipediaの記事をダウンロード
url = "http://ja.wikipedia.org/wiki/"
# 記事のリスト
wiki_name_list = ["Category:日本のポップ・グループ"]
# データを入れる
corpus = []
next_page_link = ""
tagger = MeCab.Tagger('-Owakati')

for wiki_name in wiki_name_list:
    # 指定したページの情報を取得
    response = requests.get(url+wiki_name)
    # HTMLフォーマットに変換
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    icon_part = soup.find_all("div", class_="mw-category-generated")
    
    for div_tag in icon_part:
      for ul_tag in div_tag.find_all('ul'):
        for a_tag in ul_tag.find_all('a'):
          corpus.append(a_tag.text)

    if next_page_link == "":
      next_page_part = soup.find_all("a")
      for a_tag2 in next_page_part:
        if a_tag2.text == "次のページ":
          next_page_link = "https://ja.wikipedia.org/"+a_tag2.get('href')
          
print(corpus)
print(next_page_link)

['48&46 ドリームチーム', 'EBiSSH', 'EVERYDAYS', '利用者:Facesaki/sandbox', 'あぁ!', 'アース・スター ドリーム', 'Earthmind', 'ARTERY VEIN', 'アーマーガールズ', 'Early Morning', 'RSP (音楽グループ)', 'R40', 'I WiSH', '愛内里菜&三枝夕夏', 'IQプロジェクト研究生', 'AI-SACHI', 'Aice5', 'AIS (アイドルグループ)', 'IZ4648', 'I*chip memory', 'アイドリング!!! (グループ)', 'アイドルカレッジ', 'アイドル妖怪カワユシ', 'あいまいみーまいん', 'I☆Ris', 'アイリス (by MORADOLL)', 'Ael-アエル-', '赤色のグリッター', '赤の流星', '赤マルダッシュ☆', 'AXELL', 'AQUBEE', 'あけ☆たま', 'AJI', 'AJISAI', 'AZURE (バンド)', 'AZURE♯', 'Adam (歌手グループ)', '新しい学校のリーダーズ', 'A・chi-a・chi', 'アテナ&ロビケロッツ', 'アニーポンプ', 'アニ☆ゆめ project', 'After the Rain', 'ABCHO', '危ない女の子シスターズ', '安倍なつみ&矢島舞美（℃-ute）', 'AMABILe', 'あまゆーず', 'Ami〜gas', 'あみん', 'AMOYAMO', '絢香×コブクロ', 'あややムwithエコハムず', 'AyaRuka', 'あゆみくりかまき', '嵐 (グループ)', 'アラジン (ユニット)', 'アリスインプロジェクト', 'アリス十番', 'アルケミスト (ユニット)', 'アルスマグナ (音楽ユニット)', 'ALTIMA', 'Uncle Bomb', 'AN-J', 'アンジュルム', 'Ange☆Reve', 'Anderlust', 'IA GIRL STARS', 'E-girls', 'E-sound speaker', 'ET-KING', 'EYS', 'IVOLVE', 'Yellow Cherry', 'YeLLOW Generation', 'いきもの

In [0]:
import pandas as pd
df = pd.DataFrame(data=corpus)
df.to_csv("text.txt",index=False,header=None)

##マルコフ連鎖で文章作成
https://chishiki-motomeru.com/post/20190329/

In [0]:
!pip install janome

     |████████████████████████████████| 25.1MB 85.9MB/s 


In [0]:
from janome.tokenizer import Tokenizer
import re
import random
import pickle

markov = {}
sentence = ''

#janomeを使用して文章をパースさせる。
def parse(text):
    
    t = Tokenizer()
    tokens = t.tokenize(text)
    result = []
    for token in tokens:
        result.append(token.surface)
    return(result)

#取得したファイルネームから、改行を除去した上でパースする関数を呼び出す。
#ワードリストに代入して、それを返却させる。
def get_morpheme(filename):

    with open(filename,"r",encoding = "utf_8") as f:
        text = f.read()
    text = re.sub("\n","",text)
    
    #=================================================
    #★この処理が異常に遅い
    #=================================================
    wordlist = parse(text)
    
    pickle.dump(wordlist, open('wordlist.binaryfile', 'wb'))

    return wordlist

#マルコフ辞書を生成させるmarkovは辞書型
def create_markov(wordlist):

    p1 = ""
    p2 = ""
    p3 = ""
    
    #連続した単語に続く、次の単語をサフィックスとする。単語の並びが同じであればそこに二個目のサフィックスが追加される
    for word in wordlist:
        if p1 and p2 and p3:
            if (p1,p2,p3) not in markov:
                markov[(p1,p2,p3)] = []
            markov[(p1,p2,p3)].append(word)
        p1,p2,p3 = p2,p3,word

    #プレフィックスの3つに続くサフィックスを表示する。
    #print("マルコフ辞書の中身:{}".format(markov))
    #print()

#マルコフ辞書からsentenceに文字列を格納させる。
def generate(wordlist):

    global sentence

    #3つのプレフィックスをランダムに取得して、それをプレフィックスに代入させる。
    p1,p2,p3 = random.choice(list(markov.keys()))
    print("マルコフ連鎖のキー[{},{},{}]".format(p1,p2,p3))
    print()

    count = 0

    print("【長さ】",len(wordlist))
    #while count < len(wordlist):
    while count < 100:
      if ((p1,p2,p3) in markov ) == True:
          tmp = random.choice(markov[(p1,p2,p3)])
          sentence += tmp
      p1,p2,p3 = p2,p3,tmp
      count += 1

    sentence = re.sub("^.+?","",sentence)

    if re.search(".+。",sentence):
        sentence = re.search(".+。",sentence).group()

    sentence = re.sub("」","",sentence)
    sentence = re.sub("「","",sentence)
    sentence = re.sub("　","",sentence)

    #print("【重複カット前】",sentence)

#重複した文章を取り除く。sprit("。")にすることで一文単位でリスト化できる。最終的にリストを文字列型に戻す。
#一文単位でリスト化しているので、ですますが連続した場合はここで対処できる。
def overlap():

    global  sentence

    sentence = sentence.split("。")

    #空白リストを除去する
    if "" in sentence:
        sentence.remove("")

    #この工程で一文の末尾に。を付ける。
    new = []
    for str in sentence:
        str = str + "。"
        if str == "。":
            break
        new.append(str)

    #set関数を使用して、文章の重複を取り除いている
    new = set(new)

    #set関数を使用しているので、元のリストと文章の順序が異なっている。
    #文章の順序についてはこの時点で修正したほうが良さそう
    sentence="".join(new)

if __name__ == "__main__":
    
    #①この中のparceが重い
    word_list = get_morpheme("text.txt")
    #①'ファイルから読み取り
    #word_list = pickle.load(open('wordlist.binaryfile', 'rb'))
    
    create_markov(word_list)

    while(not sentence):
        generate(word_list)
        overlap()

    print("【文章】",sentence)

マルコフ連鎖のキー[推奨,し,て]

【長さ】 11275
【文章】 談話を英訳し、世界に向けてメッセージを発信した。るし、対応はしている。その後、恒久平和への誓いと題した首相の談話を発表。その後、大野委員長が、このやりとりを理事会で報告し、二度と再び戦争の惨禍によって人々が苦しむことのない時代をつくるとの誓い、決意をお伝えするためにこの日を選んだ。


In [0]:
print(word_list)

['コブ', 'クロ', 'は', '、', '黒田', '俊介', 'と', '小渕', '健太郎', 'から', 'なる', '日本', 'の', '音楽', 'デュオ', 'で', 'ある', '。', '所属', 'レーベル', 'は', 'ワーナーミュージック・ジャパン', '。', '所属', '芸能', '事務所', 'は', 'ミノスケオフィスコブクロ', 'で', '、', '研', '音', 'と', '業務', '提携', 'し', 'て', 'いる', '。', 'その', '時', 'に', 'は', '、', '商店', '街', 'に', '約', '200', '人', 'から', '300', '人', 'の', '人だかり', 'が', 'でき', 'た', 'と', 'いう', '。', 'それ', 'を', '数', '週', '繰り返す', '内', 'に', '、', '黒田', 'から', '小渕', 'に', '「', '一緒', 'に', '組も', 'う', '」', 'と', 'ユニット', '結成', 'を', '持ちかけ', '、', '「', 'コブ', 'クロ', '」', 'が', '結成', 'さ', 'れ', 'た', '。', '因みに', '、', 'その', '時', 'に', '小渕', 'が', '黒田', 'に', '提供', 'し', 'た', '楽曲', 'が', '後', 'の', '2005', '年', 'に', 'シングル', 'として', 'リリース', 'さ', 'れ', 'た', '桜', 'で', 'ある', '。', 'ユニット', '名', 'は', '2', '人', 'の', '名字', '、', '小渕', '（', 'コブチ', '）', 'と', '黒田', '（', 'クロダ', '）', 'から', '名づけ', 'た', '。', '後', 'に', 'その', 'ネーミング', 'の', 'アイデア', 'は', 'お笑い', 'コンビ', 'の', 'FUJIWARA', '（', '藤本', '敏史', '・', '原', '西', '孝幸', '）', 'から', '来', 'て', 'いる', 'こと', 'を', '明かし', 'て',